# 01 Data Cleaning




Phase 0:

Phase 1	
	1.	Compute within / between variance of ND-GAIN at annual frequency (total, between, within, share within). Save a small table and histogram of year-to-year changes (diff1) and report per-country within SD.
	2.	Count how many countries and how many country-years show substantive ND-GAIN change; identify outliers (big diffs).
	3.	Compute per-country summary (mean, SD, number of years). Produce a short paragraph interpreting whether FE is likely to be feasible.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('/Users/leosgambato/Documents/GitHub/Capstone/data/processed/baseline_with_gain.csv')

# Clean up: drop rows where 'gain' is missing or iso3c is missing
df = df.dropna(subset=['gain', 'iso3c','sovereign_spread'])

# 1. Compute within / between variance of Gain at annual frequency

# Compute overall mean
overall_mean = df['gain'].mean()

# Compute per-country mean
country_means = df.groupby('iso3c')['gain'].mean()

# Merge country means back to df
df = df.merge(country_means.rename('country_mean'), left_on='iso3c', right_index=True)

# Between variance: variance of country means
between_var = country_means.var(ddof=0)

# Within variance: mean of per-country variances
within_var = df.groupby('iso3c')['gain'].var(ddof=0).mean()

# Total variance
total_var = df['gain'].var(ddof=0)

# Share within
share_within = within_var / total_var if total_var > 0 else np.nan

print("Variance decomposition of ND-GAIN (annual):")
print(f"  Total variance:   {total_var:.4f}")
print(f"  Between variance: {between_var:.4f}")
print(f"  Within variance:  {within_var:.4f}")
print(f"  Share within:     {share_within:.2%}")

# 2. Histogram and table of year-to-year changes (diff1), per-country within SD

# Sort for diff calculation
df = df.sort_values(['iso3c', 'year'])

# Compute year-to-year difference
df['gain_diff1'] = df.groupby('iso3c')['gain'].diff()

# Table of year-to-year changes (drop NA)
diff1_table = df[['iso3c', 'year', 'gain_diff1']].dropna()

# Save small table (first 10 rows as example)
diff1_table.head(10).to_csv('/Users/leosgambato/Documents/GitHub/Capstone/outputs/gain_diff1_sample.csv', index=False)

# Histogram of all year-to-year changes
plt.figure(figsize=(8,5))
plt.hist(diff1_table['gain_diff1'], bins=30, edgecolor='k')
plt.title('Histogram of Year-to-Year Changes in ND-GAIN')
plt.xlabel('Year-to-Year Change (diff1)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig('/Users/leosgambato/Documents/GitHub/Capstone/outputs/gain_diff1_hist.png')
plt.close()

# Per-country within SD
country_within_sd = df.groupby('iso3c')['gain'].std().rename('within_sd')
country_within_sd = country_within_sd.reset_index()
country_within_sd.head(10).to_csv('/Users/leosgambato/Documents/GitHub/Capstone/outputs/gain_within_sd_sample.csv', index=False)

# 2. Count countries and country-years with substantive ND-GAIN change; identify outliers

# Define substantive change threshold (e.g., > 1 SD of all diffs)
diff1_sd = diff1_table['gain_diff1'].std()
substantive_thresh = diff1_sd

# Count country-years with |diff1| > threshold
substantive_changes = diff1_table[np.abs(diff1_table['gain_diff1']) > substantive_thresh]
n_substantive = len(substantive_changes)
n_countries = substantive_changes['iso3c'].nunique()

print(f"\nNumber of country-years with substantive ND-GAIN change (>|{substantive_thresh:.2f}|): {n_substantive}")
print(f"Number of countries with at least one substantive change: {n_countries}")

# Identify outliers (e.g., top 5 biggest absolute diffs)
outliers = diff1_table.reindex(diff1_table['gain_diff1'].abs().sort_values(ascending=False).index).head(5)
print("\nTop 5 outlier year-to-year changes in ND-GAIN:")
print(outliers)

# 3. Per-country summary (mean, SD, number of years)
country_summary = df.groupby('iso3c').agg(
    mean_gain=('gain', 'mean'),
    sd_gain=('gain', 'std'),
    n_years=('gain', 'count')
).reset_index()

country_summary.head(10).to_csv('/Users/leosgambato/Documents/GitHub/Capstone/outputs/gain_country_summary_sample.csv', index=False)

# Short paragraph interpreting FE feasibility
print("\nInterpretation:")
print("The variance decomposition shows that a substantial share of the total variance in ND-GAIN is due to between-country differences, with within-country (over time) variance being relatively smaller. The per-country within SDs are generally low, and only a small number of country-years show substantive changes in ND-GAIN from year to year. This suggests that country fixed effects (FE) models may be feasible, but the limited within-country variation could reduce the power to detect effects of time-varying covariates. Outlier years with large changes should be checked for data quality or exceptional events.")


Variance decomposition of ND-GAIN (annual):
  Total variance:   116.0202
  Between variance: 114.1825
  Within variance:  4.5763
  Share within:     3.94%

Number of country-years with substantive ND-GAIN change (>|0.98|): 178
Number of countries with at least one substantive change: 57

Top 5 outlier year-to-year changes in ND-GAIN:
     iso3c  year  gain_diff1
2228   BGD  2014  -11.062157
2274   NGA  2014  -10.871986
2256   IND  2014   -9.909187
2278   PAK  2014   -8.339038
2270   MEX  2014   -7.692216

Interpretation:
The variance decomposition shows that a substantial share of the total variance in ND-GAIN is due to between-country differences, with within-country (over time) variance being relatively smaller. The per-country within SDs are generally low, and only a small number of country-years show substantive changes in ND-GAIN from year to year. This suggests that country fixed effects (FE) models may be feasible, but the limited within-country variation could reduce the power 

### Data cleaning, Exploratory analysis, creating lags etc. 


In [11]:
# Check for any remaining missing values
print(f"\nMissing values summary:")
missing_summary = df.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
if len(missing_summary) > 0:
    print(missing_summary)
else:
    print("No missing values in the dataset")

print("\nDataset ready for analysis!")



Missing values summary:
cpi_yoy                   267
wgi_cc                    144
wgi_ge                    144
wgi_pv                    144
wgi_rl                    144
wgi_rq                    144
wgi_va                    144
gain_diff1                 67
gdp_annual_growth_rate     66
debt_to_gdp                11
dtype: int64

Dataset ready for analysis!
